In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Define transforms for the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load the MNIST dataset
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

# Define the CNN architecture for MNIST classification
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the CNN classifier
classifier = CNNClassifier()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

# Training the CNN classifier
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = classifier(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

# Save the trained model
torch.save(classifier.state_dict(), 'mnist_cnn_classifier.pth')
print('Model saved.')

classifier.load_state_dict(torch.load('mnist_cnn_classifier.pth'))
# classifier=torch.load('mnist_cnn_classifier.pth')

# Test the trained model on a sample image
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
test_image = datasets.MNIST(root='./data', train=False, download=True, transform=test_transform)
test_label = test_image[4][1]
test_image = test_image[4][0].unsqueeze(0)

classifier.eval()
with torch.no_grad():
    output = classifier(test_image)
predicted_class = torch.argmax(output, dim=1).item()
print(f'Predicted class for the sample image: {predicted_class} {test_label}')

# Visualize the sample image
plt.imshow(test_image.squeeze(), cmap='gray')
plt.title(f'Predicted Class: {predicted_class}')
plt.axis('off')
plt.show()


Epoch 1, Batch 100, Loss: 0.736
Epoch 1, Batch 200, Loss: 0.199
Epoch 1, Batch 300, Loss: 0.129
Epoch 1, Batch 400, Loss: 0.100
Epoch 1, Batch 500, Loss: 0.098
Epoch 1, Batch 600, Loss: 0.088
Epoch 1, Batch 700, Loss: 0.070
Epoch 1, Batch 800, Loss: 0.067
Epoch 1, Batch 900, Loss: 0.065
Epoch 2, Batch 100, Loss: 0.047
Epoch 2, Batch 200, Loss: 0.060


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Define transforms for the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load the MNIST dataset
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

# Define the CNN architecture for MNIST classification
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        self.feature_maps1 = x  # Save feature maps after the first convolution
        x = x.view(-1, 64 * 7 * 7)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the CNN classifier
classifier = CNNClassifier()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

# Training the CNN classifier
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = classifier(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

# Save the trained model
torch.save(classifier.state_dict(), 'mnist_cnn_classifier.pth')
print('Model saved.')

# Test the trained model on a sample image
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
test_image = datasets.MNIST(root='./data', train=False, download=True, transform=test_transform)[0][0].unsqueeze(0)
classifier.eval()

In [ ]:
# Get the feature maps after the first convolution for the sample image
with torch.no_grad():
    output = classifier(test_image)
feature_maps1 = classifier.feature_maps1

# Visualize the feature maps
fig, axes = plt.subplots(4, 8, figsize=(16, 8))
for i in range(32):
    row = i // 8
    col = i % 8
    axes[row, col].imshow(feature_maps1.squeeze()[i], cmap='gray')
    axes[row, col].axis('off')
plt.suptitle('Feature Maps after First Convolution')
plt.show()

# Visualize the sample image
plt.imshow(test_image.squeeze(), cmap='gray')
plt.title('Sample Image')
plt.axis('off')
plt.show()

In [ ]:
# Visualize the feature maps
num_feature_maps = feature_maps1.shape[1]  # Number of output channels
fig, axes = plt.subplots(1, num_feature_maps, figsize=(16, 4))
for i in range(num_feature_maps):
    axes[i].imshow(feature_maps1.squeeze()[i], cmap='gray')
    axes[i].axis('off')
plt.suptitle('Feature Maps after First Convolution')
plt.show()

# Visualize the sample image
plt.imshow(test_image.squeeze(), cmap='gray')
plt.title('Sample Image')
plt.axis('off')
plt.show()

In [ ]:
# Get the final filters from the convolutional layers
final_filters_conv1 = classifier.conv1.weight.data
final_filters_conv2 = classifier.conv2.weight.data

# Visualize the final filters
fig, axes = plt.subplots(4, 8, figsize=(16, 8))
for i in range(32):
    row = i // 8
    col = i % 8
    axes[row, col].imshow(final_filters_conv1[i].squeeze(), cmap='gray')
    axes[row, col].axis('off')
plt.suptitle('Final Filters from Conv1 (32 Filters)')
plt.show()

# fig, axes = plt.subplots(8, 8, figsize=(16, 16))
# for i in range(64):
#     row = i // 8
#     col = i % 8
#     axes[row, col].imshow(final_filters_conv2[i].squeeze(), cmap='gray')
#     axes[row, col].axis('off')
# plt.suptitle('Final Filters from Conv2 (64 Filters)')
# plt.show()

In [ ]:
# Normalize the filter values to [0, 1]
final_filters_conv1 = (final_filters_conv1 - final_filters_conv1.min()) / (final_filters_conv1.max() - final_filters_conv1.min())
final_filters_conv2 = (final_filters_conv2 - final_filters_conv2.min()) / (final_filters_conv2.max() - final_filters_conv2.min())

# Visualize the final filters
fig, axes = plt.subplots(4, 8, figsize=(16, 8))
for i in range(32):
    row = i // 8
    col = i % 8
    axes[row, col].imshow(final_filters_conv1[i].squeeze(), cmap='gray')
    axes[row, col].axis('off')
plt.suptitle('Final Filters from Conv1 (32 Filters)')
plt.show()

fig, axes = plt.subplots(8, 8, figsize=(16, 16))
for i in range(64):
    row = i // 8
    col = i % 8
    axes[row, col].imshow(final_filters_conv2[i].squeeze(), cmap='gray')
    axes[row, col].axis('off')
plt.suptitle('Final Filters from Conv2 (64 Filters)')
plt.show()